# French translator

Talk into the mic and it'll translate your voice into French and speak it back to you.

## Setup
brew install ffmpeg

In [31]:
from pydub import AudioSegment
from pydub.playback import play
import os
from dotenv import load_dotenv
from openai import OpenAI
import json
import gradio as gr
import base64
from io import BytesIO

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-'):
    print("API key looks good so far")
else:
    print("There might be a problem with your API key?")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [32]:
def speak(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="alloy",
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

def transcribe_and_send(history, audio):
    if audio is None:
        return history
        
    # Transcribe speech
    with open(audio, "rb") as f:
        transcript = openai.audio.transcriptions.create(
            model="gpt-4o-mini-transcribe",
            file=f
        )
    english_text = transcript.text

    # Translate to French
    to_french = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Translate the user's message into French."},
            {"role": "user", "content": english_text}
        ]
    )
    french_text = to_french.choices[0].message.content.strip()

    # Say it out loud
    speak(french_text)

    # Translate French back to English (verification)
    back_to_english = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Translate the user's message into English."},
            {"role": "user", "content": french_text}
        ]
    )
    verification_text = back_to_english.choices[0].message.content.strip()
    
    # Add assistant message (translation and back-translation)
    assistant_content = (
        f"**French Translation:** {french_text}\n\n"
        f"**Back to English:** {verification_text}"
    )
    history.append({"role": "user", "content": english_text})
    history.append({"role": "assistant", "content": assistant_content})

    return history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(type="messages")
    state = gr.State([])

    audio_input = gr.Audio(sources=["microphone"], type="filepath", label="🎙️ Speak here")

    # When mic stops, transcribe and send to chat directly
    audio_input.change(fn=transcribe_and_send, inputs=[state, audio_input], outputs=chatbot)

demo.launch()

* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.


Input #0, wav, from '/var/folders/kd/tb5ggczn5gng9yy8bzfqy_ym0000gn/T/tmp8duqxua9.wav':
  Duration: 00:00:01.82, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   1.76 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 

Input #0, wav, from '/var/folders/kd/tb5ggczn5gng9yy8bzfqy_ym0000gn/T/tmpczedbbcu.wav':
  Duration: 00:00:03.26, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   3.18 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 

Input #0, wav, from '/var/folders/kd/tb5ggczn5gng9yy8bzfqy_ym0000gn/T/tmp0l_15h8w.wav':
  Duration: 00:00:01.87, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   1.74 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 

Input #0, wav, from '/var/folders/kd/tb5ggczn5gng9yy8bzfqy_ym0000gn/T/tmpt4wu5bbc.wav':
  Duration: 00:00:01.80, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   1.74 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 